In [1]:
import utils

import nltk
from nltk.corpus import stopwords
import pymorphy2

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy

In [2]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [3]:
train = pd.read_csv('data/train_data_complaints_repeats_doctors.csv')

In [4]:
train.head(3)

,Unnamed: 0,Id_Записи,Id_Пациента,Возраст,Диагноз,Жалобы,Источник_рекламы,Клиника,Код_диагноза,Пол,Услуга,Жалобы (ngramm),Жалобы (unigramm),Врач,Повторный приём
0,0,0,115819,54,Гипертензивная болезнь сердца [гипертоническая...,"на повышение ад утром до 140/90 мм.рт.ст., пер...",Другое,5,I11,2,"Прием врача-кардиолога повторный, амбулаторный",повышение_ада,повышение ада утром мм рт ст периодич головокр...,кардиолог,1
1,1,1,399973,32,Доброкачественное новообразование молочной железы,На наличие опухоли в левой молочной железе,Другое,3,D24,2,"Прием врача-онколога (маммолога), повторный, а...",наличие_опухоль левый_молочный_железо,наличие опухоль левый молочный железо,онколог,1
2,2,2,427563,72,Простой хронический бронхит,Активных жалоб нет.,Интернет,6,J41.0,2,Прием первичный врача-пульмонолога,активный_жалоба,активный жалоба,пульмонолог,0


In [5]:
complaints = train['Жалобы (unigramm)']

In [6]:
complaints.shape

(61976,)

In [7]:
train.shape

(61976, 15)

In [8]:
doctors = train['Врач'].fillna('sss')

doctors_voc, counts = np.unique(doctors, return_counts=True)
pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]

doctors[doctors == 'sss'] = pop_doctor

doctors.shape

(61976,)

In [9]:
tfidf_complaints = TfidfVectorizer(ngram_range=(1,2), min_df=10, stop_words=stopwords.words('russian'))
tfidf_complaints.fit(complaints)
list(reversed(sorted(tfidf_complaints.vocabulary_)))

['ямка',
 'яичник',
 'яичко',
 'язык',
 'ягодичный область',
 'ягодичный мышца',
 'ягодичный',
 'ягодица течение',
 'ягодица иррадиация',
 'ягодица задний',
 'ягодица бедро',
 'ягодица',
 'явление',
 'явка результат',
 'явка профилактический',
 'явка постановка',
 'явка повторный',
 'явка дообследование',
 'явка',
 'явиться цель',
 'явиться',
 'яблоко',
 'эякуляция',
 'эффект',
 'эрекция',
 'эпизодический головной',
 'эпизодический боль',
 'эпизодический',
 'эпизодически возникать',
 'эпизодически',
 'эпизод немотивированный',
 'эпизод',
 'эпигастрия эпигастральный',
 'эпигастрия право',
 'эпигастрия периодически',
 'эпигастрия натощак',
 'эпигастрия левый',
 'эпигастрия изжога',
 'эпигастрия запор',
 'эпигастрия еда',
 'эпигастрия горечь',
 'эпигастрия вздутие',
 'эпигастрия',
 'эпигастральный область',
 'эпигастральный',
 'эндоскопический динамик',
 'эндоскопический',
 'эндометрий',
 'эмоциональный стресс',
 'эмоциональный перегрузка',
 'эмоциональный нестабильность',
 'эмоциональный

In [10]:
tfidf_complaints.transform([train['Жалобы (unigramm)'][0]])

<1x4897 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [11]:
vect_doctors = CountVectorizer()
vect_doctors.fit(doctors)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
vect_doctors.transform([doctors[0]]).todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])

In [13]:
tfidf_complaints.transform(complaints)

<61976x4897 sparse matrix of type '<class 'numpy.float64'>'
	with 614938 stored elements in Compressed Sparse Row format>

In [14]:
gender = train['Пол'].copy()
gender[train['Пол'] == 1] = 0
gender[train['Пол'] == 2] = 1

In [15]:
repeats = train['Повторный приём']

In [16]:
age = train['Возраст']

In [20]:
pop_diagnoses = set(utils.get_most_popular_diagnoses(train, percent=.95))
most_pop_diagnose = scipy.stats.mode(train['Код_диагноза'])[0][0]

/home/vladvin/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [20]:
train['Код_диагноза'] = train['Код_диагноза'].apply(
    lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
)

In [48]:
def preproces_fit(data):
    # Complaints
    complaints = data['Жалобы (unigramm)']
    tfidf_complaints = TfidfVectorizer(ngram_range=(1,2), min_df=10, stop_words=stopwords.words('russian'))
    tfidf_complaints.fit(complaints)
    
    # Doctors
    doctors = train['Врач'].fillna('sss')
    doctors_voc, counts = np.unique(doctors, return_counts=True)
    pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
    doctors[doctors == 'sss'] = pop_doctor
    
    vect_doctors = CountVectorizer()
    vect_doctors.fit(doctors)
    
    # Classes
    pop_diagnoses = set(utils.get_most_popular_diagnoses(data, percent=.8))
    most_pop_diagnose = scipy.stats.mode(data['Код_диагноза'])[0][0]
    y = data['Код_диагноза'].apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return tfidf_complaints, vect_doctors, pop_doctor, y

def preprocess_transform(tfidf_complaints, vect_doctors, pop_doctor, data):
    # Complaints
    complaints = data['Жалобы (unigramm)']
    
    # Doctors
    doctors = data['Врач'].fillna(pop_doctor)
    
    # Gender
    gender = data['Пол'].copy()
    gender[data['Пол'] == 1] = 0
    gender[data['Пол'] == 2] = 1
    
    # Repeats
    repeats = data['Повторный приём']
    
    # Age
    age = data['Возраст']
    
    print(doctors.shape, gender.shape, repeats.shape, age.shape)
    
    return np.hstack([
        tfidf_complaints.transform(complaints).todense(),
        vect_doctors.transform(doctors).todense(),
        np.expand_dims(gender, axis=1),
        np.expand_dims(repeats, axis=1),
        np.expand_dims(age, axis=1)
    ])

In [101]:
class DoctorsPopularityTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        doctors = x.fillna('sss')
        doctors_voc, counts = np.unique(doctors, return_counts=True)
        self.pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
        
        return self

    def transform(self, x):
        x = x.fillna('sss')
        x[x == 'sss'] = self.pop_doctor
        
        return x
    
class GenderTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = x.copy()
        x[x == 1] = 0
        x[x == 2] = 1
        
        return np.expand_dims(x, axis=1)
    
class AgeTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.expand_dims(x, axis=1)

In [102]:
pipe = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('complaints_pipe', Pipeline([
                ('complaint_selector', ItemSelector(key='Жалобы (unigramm)')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=10, stop_words=stopwords.words('russian')))
            ])),
            ('doctor_pipe', Pipeline([
                ('doctor_selector', ItemSelector(key='Врач')),
                ('doc_pop', DoctorsPopularityTransformator()),
                ('count_vect', CountVectorizer())
            ])),
            ('gender_pipe', Pipeline([
                ('gender_selector', ItemSelector(key='Пол')),
                ('gender_transform', GenderTransformator())
            ])),
            ('age_pipe', Pipeline([
                ('age_selector', ItemSelector(key='Возраст')),
                ('age_transformator', AgeTransformator())
            ]))
        ]
    ))
])

In [103]:
pipe.fit_transform(train)

<61976x4926 sparse matrix of type '<class 'numpy.float64'>'
	with 796038 stored elements in Compressed Sparse Row format>

In [44]:
tfidf_complaints, vect_doctors, pop_doctor, y = preproces_fit(train)

/home/vladvin/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [28]:
data = preprocess_transform(tfidf_complaints, vect_doctors, pop_doctor, train)

In [29]:
data.shape

(61976, 4927)

In [30]:
data_sparse = csr_matrix(data)

In [31]:
%xdel data

In [33]:
clf = SVC()
clf.fit(data_sparse, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [34]:
test = pd.read_csv('data/test_data_complaints_repeats_doctors.csv')

In [49]:
test_data = preprocess_transform(tfidf_complaints, vect_doctors, pop_doctor, test)

(30000,) (30000,) (30000,) (30000,)


In [50]:
test_sparse = csr_matrix(test_data)

In [51]:
%xdel test_data

In [59]:
from collections import Counter

In [60]:
Counter(y).most_common()

[('J06.9', 14553),
 ('M42.1', 1949),
 ('N76.0', 1942),
 ('J00', 1379),
 ('Z32.1', 1358),
 ('N41.1', 1111),
 ('Z00.0', 1048),
 ('I11', 956),
 ('N77.1*', 956),
 ('K30', 779),
 ('J35.0', 776),
 ('M54.2', 713),
 ('N60.1', 657),
 ('N76.1', 617),
 ('M54.8', 605),
 ('M54.5', 579),
 ('Z34.0', 571),
 ('M65', 570),
 ('K29.9', 562),
 ('M42', 531),
 ('M53.8', 528),
 ('Z01.4', 518),
 ('M51', 480),
 ('Z01.8', 477),
 ('H52.1', 474),
 ('K29.5', 472),
 ('D23.9', 471),
 ('M54.4', 470),
 ('N72', 445),
 ('G90', 433),
 ('K21.0', 430),
 ('J01.0', 414),
 ('M79.1', 413),
 ('D25', 386),
 ('M51.3', 373),
 ('J04.1', 364),
 ('N86', 360),
 ('B37.3+', 358),
 ('I83.9', 318),
 ('H61.2', 312),
 ('B07', 312),
 ('L30.8', 307),
 ('E03.8', 302),
 ('N40', 296),
 ('K86.1', 295),
 ('E04.2', 293),
 ('M54.6', 292),
 ('Z34.8', 292),
 ('J18', 292),
 ('M51.8', 286),
 ('N30.0', 278),
 ('N30.2', 265),
 ('M43.9', 262),
 ('N20.0', 262),
 ('N84.0', 259),
 ('K87*', 257),
 ('K29.3', 253),
 ('N83.0', 249),
 ('Z35.8', 248),
 ('I10', 240),

In [52]:
pred = clf.predict(test_sparse)

In [56]:
np.unique(pred)

array(['I11', 'J06.9', 'N76.0'], dtype=object)